<a href="https://colab.research.google.com/github/hdgomez18/challenge_2024/blob/main/etl_withPyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **instalacion libreria**

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=52d7b90d9af64101dd254bab21bfcb61d9ab5439173ce5bce61690690c733d63
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


# Crear session de spark

In [ ]:
from pyspark.sql import SparkSession, Window
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql import types as T, functions as F


spSession = SparkSession \
            .builder \
            .appName("pyspark_code") \
            .getOrCreate()

In [ ]:
#create dataframe
data = [("Hernan",35),("Nicolas",43),('Dario',22)]
columns = ['Name','Age']
df_ex = spSession.createDataFrame(data,columns)

+-------+---+
|   Name|Age|
+-------+---+
| Hernan| 35|
|Nicolas| 43|
|  Dario| 22|
+-------+---+



In [ ]:
df_ex.where(col('Age') > 30).show()

df_ex.select(max('Age').alias('age')).show()

df_ex = df_ex.withColumn('Salary',
                 when(col('Name').isin('Hernan'),300000)
                 .when(col('Name').isin('Nicolas'),200000)
                 .when(col('Name').isin('Dario'),500000))

df_ex = df_ex.withColumn('Deparment',
                 when(col('Name').isin('Hernan'),'Sales')
                 .when(col('Name').isin('Nicolas'),'Administrative')
                 .when(col('Name').isin('Dario'),'Production'))

df_ex.groupBy('Deparment').agg(sum('Salary').alias('Salary')).show()

df_ex = df_ex.withColumn("row_numberstr", F.row_number().over(Window.partitionBy('Name',).orderBy('Name')))

w = Window.orderBy(col("row_numberstr"))

df_ex = df_ex.withColumn("id", F.row_number().over(w))

df_ex = df_ex.drop('row_numberstr')

df_ex = df_ex.select('id','Name','Age','Deparment','Salary')

df_ex = df_ex.withColumn('Deparment',regexp_replace('Deparment','Administrative','Admin'))

df_ex = df_ex.withColumnRenamed('Deparment','department')

df_ex1 = df_ex1.withColumn('id',lit(1))

df_ex1 = df_ex1.withColumn('date_hiring',lit('2023-10-01'))

df_ex1 = df_ex1.withColumn('date_hiring',col('date_hiring').cast('date'))

df_ex3 = df_ex.alias('df_a').join(df_ex1.alias('df_b'),['id'],'left').select('df_a.*','df_b.date_hiring')

df_ex3.groupBy('department').pivot('id').agg(sum('salary')).show()

+----------+------+------+------+
|department|     1|     2|     3|
+----------+------+------+------+
|Production|500000|  NULL|  NULL|
|     Sales|  NULL|300000|  NULL|
|     Admin|  NULL|  NULL|200000|
+----------+------+------+------+



In [ ]:
def return_middle(word):
    middle= int(len(word)/2)
    if middle % 2 == 0:
        return word[middle-1:middle+1]
    else: word[middle]

return_middle('hernan')

In [ ]:
def word_bk(word):
    wordbk = word[: : -1]
    print(wordbk)

word_bk('algo')

ogla



 **Carga de datos**

In [ ]:
df_depa = spSession.read.csv("departments.csv", header = 'false',sep= ';')
df_hire = spSession.read.csv("hired_employees.csv", header = 'false',sep= ';')
df_jobs = spSession.read.csv("jobs.csv", header = 'false',sep= ';')

In [ ]:
from os import truncate
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType,DoubleType,LongType
from pyspark.sql.functions import *
#delete nulls
df_depa = df_depa.where(col('_c0').isNotNull())
#replace columns names
df_depa = df_depa.withColumnRenamed('_c0','id')
df_depa = df_depa.withColumnRenamed('_c1','department')
#cast columns to integer
df_depa = df_depa.withColumn('id',col('id').cast(IntegerType()))
df_depa.show(truncate= False)
#replace columns names
df_hire = df_hire.withColumnRenamed('_c0','id')
df_hire = df_hire.withColumnRenamed('_c1','name')
df_hire = df_hire.withColumnRenamed('_c2','datetime')
df_hire = df_hire.withColumnRenamed('_c3','department_id')
df_hire = df_hire.withColumnRenamed('_c4','job_id')
#separate the rows with nulls
df_without_name = df_hire.where(col('name').isNull() | col('datetime').isNull() |  col('department_id').isNull() |  col('job_id').isNull())

df_hire_f = df_hire.where(col('name').isNotNull() | col('datetime').isNotNull() |  col('department_id').isNotNull() |  col('job_id').isNotNull())
#cast columns to integer
df_hire_f = df_hire_f.withColumn('id',col('id').cast(IntegerType()))
df_hire_f = df_hire_f.withColumn('department_id',col('department_id').cast(IntegerType()))
df_hire_f = df_hire_f.withColumn('job_id',col('job_id').cast(IntegerType()))

df_jobs.show(truncate= False)
df_f2.count()

#replace columns names
df_jobs = df_jobs.withColumnRenamed('_c0','id')
df_jobs = df_jobs.withColumnRenamed('_c1','job')
#cast columns to integer
df_jobs = df_jobs.withColumn('id',col('id').cast(IntegerType()))

CREATE DB AND TABLES

In [ ]:
db_name = "COMPANY_AAA"
spSession.sql(f"CREATE DATABASE IF NOT EXISTS {db_name}")

DataFrame[]

In [ ]:
from os.path import abspath
from pyspark.sql import SparkSession

# warehouse_location points to the default location for managed databases and tables
warehouse_location = abspath('spark-warehouse')

# Create spark session with hive enabled
spark = SparkSession \
    .builder \
    .appName("SparkByExamples.com") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

In [ ]:
#CREATE TABLE JOBS
df_jobs.write.mode('overwrite') \
    .saveAsTable("company_aaa.jobs")
#CREATE TABLE DEPARMENTS
df_depa.write.mode('overwrite') \
    .saveAsTable("company_aaa.deparments")
#CREATE TABLE hired_employees
df_hire_f.write.mode('overwrite') \
    .saveAsTable("company_aaa.hired_employees")
#CREATE TABLE hired_employees_withNulls
df_without_name.write.mode('overwrite') \
    .saveAsTable("company_aaa.employeesNulls")


save backup to parquet

In [ ]:
df_jobs.write.format("parquet").save("/content/backup/jobs.parquet")
df_depa.write.format("parquet").save("/content/backup/deparment.parquet")
df_hire_f.write.format("parquet").save("/content/backup/employees.parquet")
df_without_name.write.format("parquet").save("/content/backup/employees_null.parquet")

create function to restore table from backup

In [ ]:
def restore_table (name_table: str):
    df_backup = spark.read.parquet('/content/backup/'+name_table+'.parquet')
    return print('Please use the next dataframe: df_backup to consult the data of '+name_table)

restore_table('deparment')

Please use the next dataframe: df_backup to consult the data of deparment
